In [186]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [187]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten,Activation
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
from math import sqrt
from multiprocessing.pool import ThreadPool
import time
from sklearn.model_selection import GridSearchCV
import multiprocessing
import traceback
import tensorflow as tf
from sklearn.metrics import precision_score, make_scorer
import prettytable
import keras
import sklearn
from keras import backend as K

In [188]:
import warnings; 
warnings.filterwarnings("ignore")

In [189]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [190]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [191]:
def reshape_data(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1))

    return (x_train, x_test, y_train, y_test)

In [192]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [193]:
def create_confusion_matrix(y_true,y_pred):
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [194]:
def precision_threshold(threshold):
    def precision(y_true, y_pred):
        thresholdvalue = threshold
        # Adaptation of the "round()" used before to get the predictions. Clipping to make sure that the predicted raw values are between 0 and 1.
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), thresholdvalue), K.floatx())
        # Compute the number of true positives. Rounding in prevention to make sure we have an integer.
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        # count the predicted positives
        predicted_positives = K.sum(y_pred)
        # Get the precision ratio
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

In [195]:
def precision_new(y_true,y_pred):
    s = 0.01
    t = 0.5
    while True:
        pred = np.array(y_pred>t,int)
        res = confusion_matrix(y_true,pred)
        if res[1][1] >= 100 or t <= 0:
          break
        t = t - s
    # pred = np.array(y_pred>threshold_value,int)
    precision = metrics.precision_score(y_true,pred)
    print(" precision = ", precision)
    return precision

In [196]:
def best_params_cnn(X,Y):
    custom_precision = make_scorer(precision_new, greater_is_better=True)
    model = KerasClassifier(build_fn = create_model_c,input_shape = (X.shape[1],1))
    batch_size = [25,32,48,64,100]
    epochs = [25,50,75,100]
    param_grid = dict(epochs=epochs,batch_size=batch_size)   
    grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=-1,verbose = 1,scoring=custom_precision)
    grid_result = grid.fit(X,Y)
    batch_size, epochs = grid_result.best_params_['batch_size'],grid_result.best_params_['epochs']
    return batch_size, epochs

In [197]:
def create_model_c(input_shape):
    model = Sequential()
    model.add(Conv1D(32, 1, activation="relu", input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [198]:
def create_cnn_classification(df,column,epochs,batch_size,rate):
    df["Target"] = df[column].apply(lambda x : 1 if x >= rate else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    (x_train, x_test, y_train, y_test) = reshape_data(x_train, x_test, y_train, y_test)
    
    # model = KerasClassifier(create_model_c,input_shape = (X.shape[1],1))
    model = create_model_c((X.shape[1],1))
    history = model.fit(x_train,y_train,epochs = epochs,batch_size = batch_size, validation_data = (x_test,y_test),shuffle = False,verbose=0)

    y_pred = model.predict(x_test)
    s = 0.01
    t = 0.5
    while True:
        pred = np.array(y_pred>t,int)
        res = confusion_matrix(y_test,pred)
        if res[1][1] >= 100 or t <= 0:
          break
        t = t - s
    result = {}
    error = error_metrics(y_test, pred)
    confusion = create_confusion_matrix(y_test,pred)
    result.update(error)
    result.update(confusion)
    result.update({"threshold":t})
    return result

In [179]:
def cnn_classification(df,column,name):
    rate_of_growth = [0.01,0.02,0.03,0.04,0.05]
    solution = list()
    for t in rate_of_growth:
        print("rate = ",t)
        df["Target"] = df[column].apply(lambda x : 1 if x >= t else 0)
        X = df.drop(columns=["Target",column])
        Y = df["Target"]
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
        (x_train, x_test, y_train, y_test) = reshape_data(x_train, x_test, y_train, y_test)
        batch_size, epochs = best_params_cnn(x_train,y_train)

        result = create_cnn_classification(df,column = "Next Day Close Price GR",epochs = epochs,batch_size = batch_size,rate = t)
        result.update({"epochs":epochs,"batch_size":batch_size,"rate_of_growth":t})
        solution.append(result)
        pd.DataFrame([result]).to_csv("cnn-results/"+str(name[2:8])+str(t)+".csv",index=None)
        break
    return solution

In [180]:
# %%time
# path = "/content/drive/MyDrive/stockdata/grdata"
# threshold_value = 0.5
# for name in os.listdir(path):
#     print(name)
#     df = pd.read_csv(os.path.join(path,name))
#     df = pre_process_data(df,60)
#     df,column = dependent_column(df,"Next Day Close Price GR")
#     result = cnn_classification(df,column,name)
#     result.to_csv("cnn-results/"+str(name[2:8])+".csv",index=None)